In [ ]:
#pip install pandas

In [ ]:
#pip install openpyxl

In [ ]:
'''import requests

response = requests.get("https://search.etnews.com/etnews/search.html?kwd=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&date=0&startDate=&endDate=&detailSearch=true&category=CATEGORY1&pageSize=&search_source=&sort=1&preKwd%5B0%5D=AI")
response.status_code
'''

'import requests\n\nresponse = requests.get("https://search.etnews.com/etnews/search.html?kwd=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&date=0&startDate=&endDate=&detailSearch=true&category=CATEGORY1&pageSize=&search_source=&sort=1&preKwd%5B0%5D=AI")\nresponse.status_code\n'

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import os
import re


In [2]:

response = requests.get("https://nia.or.kr/site/nia_kor/main.do;jsessionid=6EACE24EADAB8A749EFCC1293267C284.33f82d3a14ca06361270")
html = response.text
soup = BeautifulSoup(html, 'html.parser')

data = []
items=soup.select(".article.know")
for i in range(1, 6):
    name = soup.select_one(f".article.know > ul > li:nth-child({i}) > a").attrs['title']
    category = soup.select_one(f".article.know > ul > li:nth-child({i}) > a > span.category").text
    code0 = soup.select_one(f".article.know > ul > li:nth-child({i}) > a").attrs['onclick']
    pattern = re.compile(r"'([^']*)'")
        # 모든 일치 항목을 리스트로 찾습니다.
    raw_arguments = pattern.findall(code0)
    # 추출된 인자 중 숫자로만 구성된 항목만 필터링하여 최종 리스트를 만듭니다.
    # 빈 문자열 ('')은 제외하고, isdigit()으로 순수 숫자인지 확인합니다.
    extracted_numbers = [arg for arg in raw_arguments if arg.isdigit()]
    code1 = extracted_numbers[0]
    code2 = extracted_numbers[1]
    code3 = extracted_numbers[2]
    link = f'https://nia.or.kr/site/nia_kor/ex/bbs/View.do?cbIdx={code1}&bcIdx={code2}&parentSeq={code3}'
    response = requests.get(link)
    html3 = response.text
    soup3 = BeautifulSoup(html3, 'html.parser')   
    html_string = soup3.select_one(".src>em").text
    date_parts = html_string.split('.')
    year = date_parts[0]  # '2025'
    month = date_parts[1] # '12'
    day = date_parts[2]   # '08'
    
    data.append([name,category,link,year,month,day,])
    #data.append([name,link,summary, date])

In [3]:
df3 = pd.DataFrame(data, columns=['제목','분류','링크','년','월','일'])
df3


,제목,분류,링크,년,월,일
0,[The AI Report 2025-17] 사람 중심의 AI 금융 추진 전략},지능사회 이슈분석,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,08
1,코스타리카 디지털정부 / ICT 현황 리포트},ICT 글로벌 동향,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,08
2,「AI.GOV 해외동향」 2025-4호},AI.gov,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,05
3,[The AI Report(2025-16)] 국내 제조업 경쟁력 강화를 위한 Phy...,지능사회 이슈분석,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,04
4,[Digital Insight 2025-3] AI 네트워크로의 패러다임 전환과 정책...,지능사회 이슈분석,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,03


In [4]:

full_path = 'nia.json'
new_data = df3.to_dict('records') # 새 DataFrame을 리스트 오브 딕셔너리 형태로 변환



# ----------------- JSON 이어 붙이기 및 중복 제거 로직 시작 -----------------

existing_data = []

# 1. 기존 JSON 파일 로드
if os.path.exists(full_path):
    try:
        with open(full_path, 'r', encoding='utf-8') as f:
            # 파일이 비어있지 않은지 확인 후 로드
            content = f.read()
            if content:
                existing_data = json.loads(content)
            else:
                print("기존 JSON 파일은 존재하지만 비어 있습니다. 새 데이터만 저장합니다.")
    except Exception as e:
        print(f"기존 JSON 파일 로드 중 오류 발생 ({e}). 새 데이터만 저장합니다.")
        existing_data = []

# 2. 새 데이터와 기존 데이터를 합치기
combined_data = existing_data + new_data

# 3. 중복 제거 (가장 중요한 단계)
# '링크'를 기준으로 중복 제거를 위한 Set을 생성합니다.
seen_links = set()
final_data = []

for item in combined_data:
    link = item.get('링크') # '링크' 컬럼 값을 가져옵니다.
    
    # '링크'가 None이거나 비어있지 않고, 아직 처리하지 않은 링크인 경우에만 추가
    if link and link not in seen_links:
        final_data.append(item)
        seen_links.add(link)
        
print(f"총 {len(existing_data)}개의 기존 데이터와 {len(new_data)}개의 새 데이터를 합쳤습니다.")
print(f"중복을 제거한 후 최종 데이터는 총 {len(final_data)}개입니다.")

# 4. 최종 데이터를 JSON 파일로 저장 (덮어쓰기)
# indent=4와 force_ascii=False 옵션을 유지하여 가독성 및 한글 보존
with open(full_path, 'w', encoding='utf-8') as f:
    json.dump(final_data, f, indent=4, ensure_ascii=False)

print(f"\n최종 데이터가 '{full_path}'에 성공적으로 저장되었습니다.")

총 0개의 기존 데이터와 5개의 새 데이터를 합쳤습니다.
중복을 제거한 후 최종 데이터는 총 5개입니다.

최종 데이터가 'D:/startcoding/python_crawling/04.기사 크롤링 사이트 만들기/nia.json'에 성공적으로 저장되었습니다.


In [5]:
# df2 = pd.DataFrame(data, columns=['기사명','링크','년','월','일','시','분'])
# df2['기사명'] = df2['기사명'].fillna('').str.replace(r'\\', '', regex=True)
# df2['기사명'] = df2['기사명'].str.replace('"', '＂', regex=False)

# #엑셀 저장
# #index=False 는 앞에 번호 없애기
# df2.to_excel('result.xlsx', index=False)
# full_path = 'D:/startcoding/python_crawling/04.기사 크롤링 사이트 만들기/etnews.json'
# df2.to_json(full_path, orient='records', indent=4, force_ascii=False)
df3

,제목,분류,링크,년,월,일
0,[The AI Report 2025-17] 사람 중심의 AI 금융 추진 전략},지능사회 이슈분석,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,08
1,코스타리카 디지털정부 / ICT 현황 리포트},ICT 글로벌 동향,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,08
2,「AI.GOV 해외동향」 2025-4호},AI.gov,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,05
3,[The AI Report(2025-16)] 국내 제조업 경쟁력 강화를 위한 Phy...,지능사회 이슈분석,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,04
4,[Digital Insight 2025-3] AI 네트워크로의 패러다임 전환과 정책...,지능사회 이슈분석,https://nia.or.kr/site/nia_kor/ex/bbs/View.do?...,2025,12,03
